In [ ]:
import cv2 
from scipy.misc import imresize 
import moviepy.editor as moviepy
from tqdm import tnrange, tqdm_notebook

import AsciiMosaic #cvts all chars for pixel values 
import Text2Img #cvts specifics chars from text file to values 

import matplotlib.pyplot as plt 
%matplotlib inline
# import importlib
# importlib.reload()

In [ ]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    '''a function to map values from one domain to another
    for converting the iteration index to a parameter for the crop selection'''
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [ ]:
size = [1080, 1920]

video_in = 'obama.mp4'
video_out = 'tester.avi'
orig_movie = moviepy.VideoFileClip(video_in) 

fourcc = cv2.VideoWriter_fourcc(*'XVID') #codec 
writer = cv2.VideoWriter(video_out, fourcc, orig_movie.fps, tuple(size[::-1]), False)

mosiac = Text2Img.Processor(10) #choose your processor 

In [ ]:

# with open('transcript.txt') as f: 
#     for word in f: 
#         word
# l = word.split(' ')

# # a = orig_movie.get_frame(i/orig_movie.duration)
# # a = imresize(a, 198).shape[:2]
# # (a[0] * a[1]) % l
# l.append(l.pop(2))
# l

In [ ]:
scale = 100
min_scale = 8
shift = [0, 0]

'''grab frame from video, process, resize, write to output'''
for i in tnrange(int(orig_movie.fps * orig_movie.duration), desc='progress'): #
    
    #grab the next frame and process it 
    frame = orig_movie.get_frame(i/orig_movie.duration)
    ascii_img = mosiac.process_image(imresize(frame, 200), shift = i//3)
    ascii_shp = ascii_img.shape
    center = [int(x/2) for x in ascii_shp]
    min_dims = [int(x*min_scale/100) for x in ascii_shp]
    
    '''adjust frame parameters'''
    #frames to pan over
    start1, end1 = 100, 300
    start2, end2 = 400, 500
    start3, end13 = 500, 700 
    
    i+= 200
    if i > start1 and i <= end1: 
        '''scale down and push to left corner'''
        scale = int(translate(i, start1, end1, 100, min_scale))
        shift[0] = int(translate(i, start1, end1, 0, -(size[0]/2 - min_dims[0])))
        shift[1] = int(translate(i, start1, end1, 0, -(size[1]/2 - min_dims[1])))
         
    elif i > start2 and i <= end2: 
        '''keep scale and move to right corner'''
        shift[1] = int(translate(i, start2, end2, -(size[1]/2 - min_dims[1]), size[1]/2 - min_dims[1]))
    
    if i > start3 and i <= end13: 
        '''scale up and move to origin'''
        scale = int(translate(i, start3, end13, min_scale, 100))
        shift[0] = int(translate(i, start3, end13, -(ascii_shp[0]/2 - min_dims[0]), 0))
        shift[1] = int(translate(i, start3, end13, (ascii_shp[1]/2 - min_dims[1]), 0))

    #crop the selection determined by the above parameters 
    new_center = [sum(x) for x in zip(center, shift)]
    rect_size_x, rect_size_y = [int(x/2*scale/100) for x in ascii_shp]

    cropped = ascii_img[(new_center[0] - rect_size_x):(new_center[0] + rect_size_x),
                   (new_center[1] - rect_size_y):(new_center[1] + rect_size_y)]

#     print (shift, center, new_center)
#     print (scale, rect_size_x, rect_size_y)
#     print (ascii_shp, cropped.shape)
#     print ((new_center[0] - rect_size_x), (new_center[0] + rect_size_x))
#     print ((new_center[1] - rect_size_y), (new_center[1] + rect_size_y))
#     print()
    writer.write(imresize(cropped, size).astype('u1'))
    
writer.release()

In [ ]:
#for stopping early 
writer.release()
cv2.imwrite('test_cropped.png', cropped)
cv2.imwrite('test_full.png', ascii_img)